# European DATE Practical Exercise
This notebook snatches a ZIP containing boundary shapefiles describing the [European DATE](https://odin.tradoc.army.mil/DATE/83a94dfd68f15bda5615c97fd8ac7358) area, then adds them to a Folium map using GeoPandas. Check out [the resulting map here](http://gusmann.github.io/Euro-DATE/).


In [1]:
# third party imports
import zipfile
from pathlib import Path
from requests import Session
import io
import geopandas as gpd
from folium import Map
import folium
import matplotlib.colors as mcolors

client = Session()

zipfile_uri = "https://g2webcontent.z2.web.core.usgovcloudapi.net/OEE/DATE/DATE_E_Boundary_Shapefiles.zip"
response = client.get(zipfile_uri)
target_directory = Path('DATE_shapefiles')

In [2]:
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

In [3]:
zip_file.extractall(target_directory)

In [7]:
dfs = {}
for filename in target_directory.glob('*.shp'):
    dfs[filename] = gpd.read_file(filename)
    print(filename)

DATE_shapefiles/DATE_Europe_Boundaries.shp
DATE_shapefiles/DATE_Europe_Boundaries_Lines.shp
DATE_shapefiles/DATE_Europe_Capital_Cities.shp


In [6]:
filename

PosixPath('DATE_shapefiles/DATE_Europe_Capital_Cities.shp')

In [107]:
boundaries = dfs[target_directory / "DATE_Europe_Boundaries.shp"].drop(columns=['REGION','CODE','STATUS'])
boundary_lines = dfs[target_directory / "DATE_Europe_Boundaries_Lines.shp"].drop(columns=['Version','Status'])
capital_cities = dfs[target_directory / "DATE_Europe_Capital_Cities.shp"].drop(columns=['region','version','status'])

In [108]:
capital_cities.country.value_counts()

country
Framland    1
Arnland     1
Bothnia     1
Pirtuni     1
Torrike     1
Donovia     1
Otso        1
Name: count, dtype: int64

In [109]:
boundary_lines.Boundary.value_counts()

Boundary
Bothnia / Framland     2
Donovia / Otso         1
Donovia / Pirtuni      1
Pirtuni / [foreign]    1
Arnland / Torrike      1
Framland / Torrike     1
Bothnia / Otso         1
Bothnia / Torrike      1
Otso / Torrike         1
Torrike / [foreign]    1
Donovia / [foreign]    1
Name: count, dtype: int64

In [110]:
boundaries.NAME_SHORT.value_counts()

NAME_SHORT
Donovia     1
Pirtuni     1
Arnland     1
Torrike     1
Bothnia     1
Framland    1
Otso        1
Name: count, dtype: int64

In [111]:
# exclude Donovia since it'd disproportionately skew the map towards the east
all_geom_except_donovia = boundaries[~boundaries.NAME_SHORT.str.contains('Donovia')].geometry

In [112]:
# estimate and store the most appropriate Universal Transverse Mercator (UTM) Coordinate Reference System (CRS) for the geom
utm_crs = all_geom_except_donovia.estimate_utm_crs()
# convert to UTM CRS and calculate centroid of all shapes *in the most apt 2d plane*
point = all_geom_except_donovia.to_crs(utm_crs).centroid.union_all().centroid
# convert to geoseries
point_gs = gpd.GeoSeries([point], crs=utm_crs)
# Convert the GeoSeries to EPSG:4326, grab the single point
center_point = point_gs.to_crs(epsg=4326)[0]

In [121]:
import branca.colormap as cm
from folium.plugins import Fullscreen, Draw, MousePosition, MeasureControl

all_categories = set(capital_cities.country) | set(boundary_lines.Boundary) | set(boundaries.NAME_SHORT)
num_colors = len(all_categories)
colormap = cm.linear.Set1_09.scale(0, num_colors - 1)  

category_colors = {category: colormap(i) for i, category in enumerate(all_categories)}
capital_cities['color'] = capital_cities.country.map(category_colors)
boundary_lines['color'] = boundary_lines.Boundary.map(category_colors)
boundaries['color'] = boundaries.NAME_SHORT.map(category_colors)

In [114]:
capital_cities

,name,country,geometry,color
0,Freja,Framland,POINT (20.23999 63.82999),#40928bff
1,Alsstad,Arnland,POINT (13.0333 55.58334),#d66f81ff
2,Brahea,Bothnia,POINT (24.93218 60.17751),#c68eabff
3,Kiev,Pirtuni,POINT (30.51468 50.43531),#e77326ff
4,Tyr,Torrike,POINT (18.09539 59.35271),#b05b39ff
5,Moscow,Donovia,POINT (37.61358 55.75411),#f0e231ff
6,Otavia,Otso,POINT (26.95115 60.47163),#f282bdff


In [115]:
boundaries

,NAME_FULL,NAME_SHORT,DEMONYM,LANGUAGE,geometry,color
0,United Republics of Donovia,Donovia,Donovians,None,"MULTIPOLYGON (((132.44899 42.8454, 132.44988 4...",#f0e231ff
1,Republic of Pirtuni,Pirtuni,Pirtunians,None,"MULTIPOLYGON (((29.80747 45.61433, 29.73097 45...",#e77326ff
2,Republic of Arnland,Arnland,Arns,Arnish,"MULTIPOLYGON (((14.63742 58.09763, 14.64428 58...",#d66f81ff
3,Republic of Torrike,Torrike,Torrikans,Torrikan,"MULTIPOLYGON (((28.15916 69.91388, 28.19722 69...",#b05b39ff
4,Bothnian Democratic Republic,Bothnia,Bothnians,Bothnian,"MULTIPOLYGON (((27.49792 67.15104, 27.49828 67...",#c68eabff
5,Republic of Framland,Framland,Frams,Framish,"MULTIPOLYGON (((23.47157 66.94532, 23.47579 66...",#40928bff
6,Republic of Otso,Otso,Otsans,Otsonian,"POLYGON ((27.80783 60.5464, 27.80666 60.54861,...",#f282bdff


In [126]:
m = Map(tiles='CartoDB positron', location=(center_point.y, center_point.x), zoom_start=5)

capital_cities_lg = folium.FeatureGroup(name='Capital Cities').add_to(m)
boundary_lines_lg = folium.FeatureGroup(name='Boundary Lines').add_to(m)
countries_lg = folium.FeatureGroup(name='Countries').add_to(m)

capital_cities.explore(column='name', m=capital_cities_lg, legend=False, marker_type='circle', color=capital_cities['color'].values)
boundary_lines.explore(column='Boundary', m=boundary_lines_lg, legend=False, color=boundary_lines['color'].values)
boundaries.explore(column='NAME_SHORT', m=countries_lg, color=boundaries['color'].values)
folium.LayerControl().add_to(m)
Fullscreen().add_to(m)
MousePosition().add_to(m)
Draw().add_to(m)
MeasureControl().add_to(m)

m.save('index.html')

In [ ]:
m